In [1]:
import numpy as np
import torch

In [31]:
x = torch.tensor([[0., 1.], [1., 0.]], dtype=torch.complex128)
y = torch.tensor([[0., -1.j], [1.j, 0.]], dtype=torch.complex128)
z = torch.tensor([[1., 0.], [0., -1.]], dtype=torch.complex128)
i2 = torch.eye(2, dtype=torch.complex128)

xx = torch.kron(x, x)
yy = torch.kron(y, y)
rr = (xx + yy) / 2.
rr2 = torch.matmul(rr, rr)
d = torch.tensor([[1., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 1.]], dtype=torch.complex128)
def Rrr(theta):
    if isinstance(theta, torch.Tensor):
        th = theta
    else:
        th = torch.tensor(theta)
        
    return d + torch.cos(th) * rr2 - 1.j * torch.sin(th) * rr

cx = torch.tensor([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 0., 1.], [0., 0., 1., 0.]], dtype=torch.complex128)

def Rt(t, theta):
    if isinstance(theta, torch.Tensor):
        th2 = theta / 2.
    else:
        th2 = torch.tensor(theta / 2.)
    return torch.cos(th2) * i2 - 1.j * torch.sin(th2) * t

def U2(phi, lamb):
    return torch.chain_matmul(Rt(z, phi), Rt(y, np.pi / 2.), Rt(z, lamb))

In [32]:
class DecompositionFinder(torch.nn.Module):
    def __init__(self):
        super(DecompositionFinder, self).__init__()
        
        # u2-cx-rz-cx
        # u2-+--ry-+
        self.u2_0_0_lambda = torch.nn.Parameter(torch.empty(1))
        self.u2_0_0_phi = torch.nn.Parameter(torch.empty(1))
        self.u2_1_0_lambda = torch.nn.Parameter(torch.empty(1))
        self.u2_1_0_phi = torch.nn.Parameter(torch.empty(1))
        self.rz_0_0_phi = torch.nn.Parameter(torch.empty(1))
        self.ry_1_0_phi = torch.nn.Parameter(torch.empty(1))
        
        self.ansatz = torch.chain_matmul(
                torch.kron(U2(self.u2_0_0_phi, self.u2_0_0_lambda), torch.eye(2)),
                torch.kron(torch.eye(2), U2(self.u2_1_0_phi, self.u2_1_0_lambda)),
                cx,
                torch.kron(Rt(z, self.rz_0_0_phi), torch.eye(2)),
                torch.kron(torch.eye(2), Rt(y, self.ry_1_0_phi)),
                cx
        )
        
    def forward(self, theta):
        return 1. - torch.abs(torch.trace(torch.matmul(Rrr(theta), self.ansatz)) / 4.)
        

In [33]:
model = DecompositionFinder()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for _ in range(10):
    def closure():
        optimizer.zero_grad()
        pred = model(0.2)
        print(pred)
        pred.backward(retain_graph=True)
        return pred
        
    optimizer.step(closure)

tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)
tensor(0.7512, dtype=torch.float64, grad_fn=<RsubBackward1>)


In [41]:
u2_0_0_phi = torch.empty(1, dtype=torch.float64, requires_grad=True)
u2_0_0_lambda = torch.empty(1, dtype=torch.float64, requires_grad=True)
u2_1_0_phi = torch.empty(1, dtype=torch.float64, requires_grad=True)
u2_1_0_lambda = torch.empty(1, dtype=torch.float64, requires_grad=True)
rz_0_0_phi = torch.empty(1, dtype=torch.float64, requires_grad=True)
ry_1_0_phi = torch.empty(1, dtype=torch.float64, requires_grad=True)

def abstrace(theta):
    ansatz = torch.chain_matmul(
       torch.kron(U2(u2_0_0_phi, u2_0_0_lambda), i2),
       torch.kron(i2, U2(u2_1_0_phi, u2_1_0_lambda)),
       cx,
       torch.kron(Rt(z, rz_0_0_phi), i2),
       torch.kron(i2, Rt(y, ry_1_0_phi)),
       cx
    )

    return torch.abs(torch.trace(torch.matmul(Rrr(theta), ansatz)) / 4.)

def loss_function(at):
    return torch.tensor(1., dtype=torch.float64) - at

In [42]:
at = abstrace(0.4)
loss = loss_function(at)

In [46]:
optimizer = torch.optim.Adam([u2_0_0_phi, u2_0_0_lambda, u2_1_0_phi, u2_1_0_lambda, rz_0_0_phi, ry_1_0_phi], lr=0.6)
#optimizer = torch.optim.Adam([u2_0_0_phi], lr=0.1)

for _ in range(100):
    at = abstrace(0.4)
    loss = loss_function(at)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(loss.detach().numpy().item())

0.5213188736114618
0.718103472611408
0.5425749576121361
0.5491768836469255
0.6149893610737223
0.570923717624815
0.5137083906476951
0.5285905034548213
0.5653698640271216
0.55834137046717
0.5240514047047469
0.5105037476628065
0.5293814296540195
0.5442733170864641
0.5339353649824978
0.5145304359927486
0.5112135599592129
0.5236787889873057
0.5303903296967213
0.5223100237073496
0.5113729125842796
0.5114525612347783
0.5193518473488696
0.5217793573012213
0.5154712673581117
0.5096643634636907
0.5115357692714453
0.5163241730941459
0.5160315632699468
0.5112775068604964
0.5089316197681915
0.5113416502230814
0.5135496205361392
0.5117741130808408
0.5087528202489404
0.5086418355052735
0.5105367674367197
0.510649521016888
0.5085614940918004
0.5072956201090767
0.5081115852033993
0.508809606779286
0.5077477623797515
0.5063353163167108
0.5062997751109974
0.5068709045371076
0.5064466966282066
0.5053305350354279
0.5049101467865955
0.5052033211394042
0.5050610507850695
0.5042745042523571
0.5037698868370812

In [38]:
x = torch.tensor(1., requires_grad=True)
optimizer = torch.optim.Adam([x], lr=0.1)
def loss_fct(x):
    return x * x

for _ in range(10):
    def closure():
        optimizer.zero_grad()
        loss = loss_fct(x)
        print(loss.detach().numpy().item())
        loss.backward()
        return loss
    
    optimizer.step(closure)

1.0
0.8100000619888306
0.640659749507904
0.4922233521938324
0.3647424578666687
0.2580271363258362
0.1715918928384781
0.10460098832845688
0.05582057312130928
0.023588238283991814


In [39]:
l = loss_fct(x)
print(l.backward())

None


In [40]:
l.grad

<ipython-input-40-97b42495692b>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  l.grad
